In [1]:
import instructor
from openai import OpenAI

from enum import Enum
from pydantic import BaseModel, Field
from typing_extensions import Literal

In [2]:
client = instructor.patch(OpenAI())

# Tip: Do not use auto() as they cast to 1,2,3,4
class House(Enum):
    Gryffindor = "gryffindor"
    Hufflepuff = "hufflepuff"
    Ravenclaw = "ravenclaw"
    Slytherin = "slytherin"


class Character(BaseModel):
    age: int
    name: str
    house: House

    def say_hello(self):
        print(
            f"Hello, I'm {self.name}, I'm {self.age} years old and I'm from {self.house.value.title()}"
        )

In [3]:
resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

{'age': 11, 'name': 'Harry Potter', 'house': <House.Gryffindor: 'gryffindor'>}

In [4]:
resp.say_hello()

Hello, I'm Harry Potter, I'm 11 years old and I'm from Gryffindor


In [5]:
class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

{'age': 17, 'name': 'Harry Potter', 'house': 'Gryffindor'}

### Arbitrary propreties

In [6]:
from typing import List


class Property(BaseModel):
    key: str = Field(description="Must be snake case")
    value: str


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]
    properties: List[Property]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Snape from Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

{'age': 38,
 'name': 'Severus Snape',
 'house': 'Slytherin',
 'properties': [{'key': 'occupation',
   'value': 'Professor at Hogwarts, Head of Slytherin House'},
  {'key': 'patronus', 'value': 'Doe'},
  {'key': 'loyalty', 'value': 'Albus Dumbledore'},
  {'key': 'skills', 'value': 'Potions master, Occlumency, Legilimency'}]}

### Limiting the length of lists

In [7]:
class Property(BaseModel):
    index: str = Field(..., description="Monotonically increasing ID")
    key: str = Field(description="Must be snake case")
    value: str


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]
    properties: List[Property] = Field(
        ...,
        description="Numbered list of arbitrary extracted properties, should be exactly 5",
    )


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Snape from Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

{'age': 38,
 'name': 'Severus Snape',
 'house': 'Slytherin',
 'properties': [{'index': '1',
   'key': 'role_at_hogwarts',
   'value': 'Potions Master'},
  {'index': '2', 'key': 'patronus', 'value': 'Doe'},
  {'index': '3', 'key': 'loyalty', 'value': 'Dumbledore'},
  {'index': '4', 'key': 'spy_for_the_order', 'value': 'True'},
  {'index': '5', 'key': 'double_agent', 'value': 'Yes'}]}

### Defining multiple entities

In [8]:
from typing import Iterable


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Five characters from Harry Potter"}],
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

age=17 name='Harry Potter' house='Gryffindor'
age=17 name='Hermione Granger' house='Gryffindor'
age=17 name='Ron Weasley' house='Gryffindor'
age=17 name='Draco Malfoy' house='Slytherin'
age=16 name='Luna Lovegood' house='Ravenclaw'


In [9]:
class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Five characters from Harry Potter"}],
    stream=True, # with streaming
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

age=17 name='Harry Potter' house='Gryffindor'
age=17 name='Hermione Granger' house='Gryffindor'
age=17 name='Ron Weasley' house='Gryffindor'
age=16 name='Draco Malfoy' house='Slytherin'
age=15 name='Luna Lovegood' house='Ravenclaw'


### Defining relationships

In [10]:
class Character(BaseModel):
    id: int
    name: str
    friends_array: List[int] = Field(description="Relationships to their friends using the id")


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "5 kids from Harry Potter"}],
    stream=True,
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

id=1 name='Harry Potter' friends_array=[2, 3, 4, 5]
id=2 name='Hermione Granger' friends_array=[1, 3, 4]
id=3 name='Ron Weasley' friends_array=[1, 2, 4, 5]
id=4 name='Ginny Weasley' friends_array=[1, 2, 3]
id=5 name='Draco Malfoy' friends_array=[1, 3]


### Maybe pattern

In [16]:
from typing import Optional

class Character(BaseModel):
    age: int
    name: str

class MaybeCharacter(BaseModel):
    result: Optional[Character] = Field(default=None, description="Character information such as age and name")
    error: bool = Field(default=False, description="Whether there is an error or not")
    message: Optional[str]

In [19]:
def extract(content: str) -> MaybeCharacter:
    return client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=MaybeCharacter,
        messages=[
            {"role": "user", "content": f"`{content}`"},
        ],
    )

In [20]:
extract("Harry Potter")


MaybeCharacter(result=Character(age=33, name='Harry Potter'), error=False, message="Harry Potter is a fictional character and the main protagonist in J.K. Rowling's Harry Potter series.")

In [21]:
user = extract("404 error")

if user.error:
    raise ValueError(user.message)

ValueError: 404 error